In this notebook we will train a deep learning model using all the data available !
* preprocessing : I encoded the smiles of all the train & test set and saved it [here](https://www.kaggle.com/datasets/ahmedelfazouan/belka-enc-dataset) , this may take up to 1 hour on TPU.
* Training & Inference : I used a simple 1dcnn model trained on 20 epochs.

How to improve :
* Try a different architecture : I'm able to get an LB score of 0.604 with minor changes on this architecture.
* Try another model like Transformer, or LSTM.
* Train for more epochs.
* Add more features like a one hot encoding of bb2 or bb3.
* And of course ensembling with GBDT models.

In [7]:
!pip install fastparquet -q


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import gc
import os
import pickle
import random
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score as APS

In [3]:
class CFG:

    PREPROCESS = False
    EPOCHS = 20
    BATCH_SIZE = 4096
    LR = 1e-3
    WD = 0.05

    NBR_FOLDS = 15
    SELECTED_FOLDS = [0]

    SEED = 2024

In [4]:
import tensorflow as tf
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

set_seeds(seed=CFG.SEED)

D0805 14:07:57.898085871      15 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0805 14:07:57.898109521      15 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0805 14:07:57.898113045      15 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0805 14:07:57.898115792      15 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0805 14:07:57.898118184      15 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0805 14:07:57.898120764      15 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0805 14:07:57.898123180      15 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0805 14:07:57.

In [5]:
import tensorflow as tf

# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu="local") # "local" for 1VM TPU
    strategy = tf.distribute.TPUStrategy(tpu)
    print("Running on TPU")
    print("REPLICAS: ", strategy.num_replicas_in_sync)
except tf.errors.NotFoundError:
    print("Not on TPU")

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU

# Preprocessing

In [25]:
from tqdm import tqdm
import pandas as pd
import numpy as np

# train_raw = pd.read_parquet('/kaggle/input/leash-BELKA/train.parquet')
# smiles = train_raw[train_raw['protein_name']=='BRD4']['molecule_smiles'].values
# assert (smiles!=train_raw[train_raw['protein_name']=='HSA']['molecule_smiles'].values).sum() == 0
# assert (smiles!=train_raw[train_raw['protein_name']=='sEH']['molecule_smiles'].values).sum() == 0
import pandas as pd

df = pd.read_csv('/kaggle/input/dataset/train.csv')
df2 = pd.read_csv('/kaggle/input/dataset/test.csv')
def encode_smile(smile):
    tmp = [enc[i] for i in smile]
    tmp = tmp + [0]*(145-len(tmp))
    return np.array(tmp).astype(np.uint8)


df = df[['Smiles', 'pIC50']]
smiles = df['Smiles'].values
smiles2 = df2['Smiles'].values
enc = {}
for i in df['Smiles']:
    for j in list(i):
        if enc.get(j, None) == None:
            enc[j] = len(enc) + 1
            
smiles_enc = [encode_smile(smile) for smile in tqdm(smiles)]
smiles_enc = np.stack(smiles_enc)
smiles_enc2 = [encode_smile(smile) for smile in tqdm(smiles2)]
smiles_enc2 = np.stack(smiles_enc2)
train = pd.DataFrame(smiles_enc, columns = [f'enc{i}' for i in range(145)])
test = pd.DataFrame(smiles_enc2, columns = [f'enc{i}' for i in range(145)])
train['pIC50'] = df['pIC50']

100%|██████████| 113/113 [00:00<00:00, 47744.17it/s]


# Modeling

In [114]:
def my_model():
    def log_cosh_loss(y_true, y_pred):
        return tf.reduce_mean(tf.math.log(tf.math.cosh(y_pred - y_true)))
    
    with strategy.scope():
        INP_LEN = 145
        NUM_FILTERS = 32
        hidden_dim = 128

        inputs = tf.keras.layers.Input(shape=(INP_LEN,), dtype='int32')
        x = tf.keras.layers.Embedding(input_dim=len(enc), output_dim=hidden_dim, input_length=INP_LEN, mask_zero=True)(inputs)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS, kernel_size=3, activation='relu', padding='valid', strides=1)(x)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS*2, kernel_size=3, activation='relu', padding='valid', strides=1)(x)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS*3, kernel_size=3, activation='relu', padding='valid', strides=1)(x)
        x = tf.keras.layers.GlobalMaxPooling1D()(x)

        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)
        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)
        x = tf.keras.layers.Dense(512, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)

        outputs = tf.keras.layers.Dense(1)(x)

        model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
        optimizer = tf.keras.optimizers.Adam(learning_rate=CFG.LR, weight_decay=CFG.WD)
        loss = 'mean_absolute_error'  # 회귀 문제에 적합한 손실 함수
        # mean_squared_logarithmic_error
        # mean_squared_error
        # log_cosh_loss
        # huber_loss
        # mean_absolute_error
        model.compile(
            loss=loss,
            optimizer=optimizer,
            metrics=['mean_absolute_error']  # 회귀 문제에 적합한 평가 지표
        )
        return model

# Train & Inference

In [107]:
import numpy as np

def pIC50_to_IC50(pic50_values):
    return 10 ** (9 - pic50_values)

def calculate_score(y_true, y_pred):
    y_true = pIC50_to_IC50(y_true)
    y_pred = pIC50_to_IC50(y_pred)
    
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    norm_rmse = rmse / np.ptp(y_true)  # np.ptp(y_true)는 y_true의 최대값 - 최소값
    
    def ic50_to_pic50(ic50):
        return -np.log10(ic50 * 1e-9)  # IC50(nM) -> IC50(M) 변환 후 log10

    # Correct Ratio (B) 계산
    y_true_pic50 = ic50_to_pic50(y_true)
    y_pred_pic50 = ic50_to_pic50(y_pred)
    correct = np.abs(y_true_pic50 - y_pred_pic50) <= 0.5
    ratio = np.mean(correct)
    
    # 최종 Score 계산
    score = 0.5 * (1 - min(norm_rmse, 1)) + 0.5 * ratio
    return score

In [115]:
from sklearn.model_selection import KFold

# KFold 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)


all_preds = []
avg_score = 0
for fold, (train_idx, valid_idx) in enumerate(kf.split(train)):
    X_train = train.loc[train_idx, FEATURES]
    y_train = train.loc[train_idx, TARGETS]
    X_val = train.loc[valid_idx, FEATURES]
    y_val = train.loc[valid_idx, TARGETS]

    es = tf.keras.callbacks.EarlyStopping(patience=5, monitor="val_loss", mode='min', verbose=1)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', filepath=f"model-{fold}.h5",
                                                        save_best_only=True, save_weights_only=True,
                                                    mode='min')
    reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.05, patience=3, verbose=1)
    model = my_model()
    history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=20,
            callbacks=[checkpoint, reduce_lr_loss, es],
            batch_size=CFG.BATCH_SIZE,
            verbose=1,
        )
    model.load_weights(f"model-{fold}.h5")
    oof = model.predict(X_val, batch_size = 2*CFG.BATCH_SIZE)
    cv_score = calculate_score(y_val, oof)
    avg_score += cv_score
    print('fold :', fold, 'CV score =', cv_score)
    
    preds = model.predict(test, batch_size = 2*CFG.BATCH_SIZE)
    all_preds.append(preds)

preds = np.mean(all_preds, 0)

Epoch 1/20


2024-08-05 16:22:04.287378: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2024-08-05 16:22:04.483697: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


1/1 [==============================] - ETA: 0s - loss: 7.5134 - mean_absolute_error: 7.5134

2024-08-05 16:22:08.441740: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2024-08-05 16:22:08.563228: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


1/1 [==============================] - 8s 8s/step - loss: 7.5134 - mean_absolute_error: 7.5134 - val_loss: 7.3427 - val_mean_absolute_error: 7.3427 - lr: 0.0010
Epoch 2/20
1/1 [==============================] - 1s 641ms/step - loss: 7.3437 - mean_absolute_error: 7.3437 - val_loss: 7.0314 - val_mean_absolute_error: 7.0314 - lr: 0.0010
Epoch 3/20
1/1 [==============================] - 1s 627ms/step - loss: 7.0304 - mean_absolute_error: 7.0304 - val_loss: 6.4128 - val_mean_absolute_error: 6.4128 - lr: 0.0010
Epoch 4/20
1/1 [==============================] - 1s 598ms/step - loss: 6.4108 - mean_absolute_error: 6.4108 - val_loss: 5.2603 - val_mean_absolute_error: 5.2603 - lr: 0.0010
Epoch 5/20
1/1 [==============================] - 1s 625ms/step - loss: 5.2565 - mean_absolute_error: 5.2565 - val_loss: 3.2340 - val_mean_absolute_error: 3.2340 - lr: 0.0010
Epoch 6/20
1/1 [==============================] - 1s 582ms/step - loss: 3.2280 - mean_absolute_error: 3.2280 - val_loss: 0.8554 - val_mean_

...

1/1 [==============================] - ETA: 0s - loss: 7.5293 - mean_absolute_error: 7.5293

2024-08-05 16:23:27.445886: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2024-08-05 16:23:27.544765: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


1/1 [==============================] - 8s 8s/step - loss: 7.5293 - mean_absolute_error: 7.5293 - val_loss: 7.2369 - val_mean_absolute_error: 7.2369 - lr: 0.0010
Epoch 2/20
1/1 [==============================] - 1s 595ms/step - loss: 7.3423 - mean_absolute_error: 7.3423 - val_loss: 6.9026 - val_mean_absolute_error: 6.9026 - lr: 0.0010
Epoch 3/20
1/1 [==============================] - 1s 551ms/step - loss: 7.0051 - mean_absolute_error: 7.0051 - val_loss: 6.2490 - val_mean_absolute_error: 6.2490 - lr: 0.0010
Epoch 4/20
1/1 [==============================] - 1s 601ms/step - loss: 6.3492 - mean_absolute_error: 6.3492 - val_loss: 5.0491 - val_mean_absolute_error: 5.0491 - lr: 0.0010
Epoch 5/20
1/1 [==============================] - 1s 609ms/step - loss: 5.1437 - mean_absolute_error: 5.1437 - val_loss: 2.9618 - val_mean_absolute_error: 2.9618 - lr: 0.0010
Epoch 6/20
1/1 [==============================] - 1s 652ms/step - loss: 3.0550 - mean_absolute_error: 3.0550 - val_loss: 0.8936 - val_mean_

2024-08-05 16:23:37.443530: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2024-08-05 16:23:37.513242: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


1/1 [==============================] - 2s 2s/step
fold : 4 CV score = 0.6887225569780718
1/1 [==============================] - 1s 806ms/step


In [116]:
print(avg_score / 5)
# mse 0.5927130819266196 0.6423098446674433_fold5
# log_cosh_loss 0.5821399477622217
# mean_squared_logarithmic_error 0.5633181990480502
# huber_loss 0.576905494263006 0.6219190091656438_fold5
# mean_absolute_error 0.5704392483575251 0.6440441485596432_fold5

0.6128430427354579


In [117]:
sub = pd.read_csv('/kaggle/input/dataset/sample_submission.csv')
sub['IC50_nM'] = pIC50_to_IC50(preds)
sub.to_csv('sub.csv', index=False)
sub

,ID,IC50_nM
0,TEST_000,247.044418
1,TEST_001,258.041046
2,TEST_002,140.176926
3,TEST_003,210.592743
4,TEST_004,234.703018
...,...,...
108,TEST_108,251.611298
109,TEST_109,187.687195
110,TEST_110,43.605057
111,TEST_111,217.942398


# Submission